# FRACKTAL: Symbolic Tree Visualization

## Interactive Visualization of Recursive Symbolic Ontology Trees

This notebook demonstrates how FRACKTAL creates and visualizes its recursive symbolic ontology trees. You'll see how data is transformed into symbolic representations and how the tree structure emerges.

### What You'll See:
- **Symbolic Tree Construction** - How data becomes symbols
- **Tree Visualization** - Interactive plots of the RSO structure
- **Pattern Analysis** - How repeating patterns are identified
- **Depth Analysis** - Tree depth and complexity metrics

In [ ]:
# Install required packages if not already installed
import sys
import subprocess

def install_package(package):
    try:
        __import__(package)
        print(f"✅ {package} already installed")
    except ImportError:
        print(f"📦 Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Install visualization dependencies
install_package('matplotlib')
install_package('seaborn')
install_package('networkx')
install_package('plotly')

print("\n🚀 All visualization dependencies ready!")

In [ ]:
# Import FRACKTAL and visualization libraries
import sys
import os
sys.path.append('..')

from fracktal import RecursiveFRSOE
from fracktal.models import SymbolicTree, CodexMap
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

print("✅ FRACKTAL and visualization libraries imported successfully!")

## 1. Create Sample Data for Tree Visualization

Let's create some interesting data that will generate clear tree structures.

In [ ]:
# Create sample data with clear patterns
sample_data = {
    'repetitive_text': 'Hello World! Hello World! Hello World! Hello World! Hello World!',
    'structured_data': 'Name: John, Age: 30, City: NYC, Name: Jane, Age: 25, City: LA, Name: Bob, Age: 35, City: SF',
    'code_snippet': 'def function(): return True; def function(): return True; def function(): return True',
    'nested_structure': '{"user": {"name": "John", "age": 30}, "user": {"name": "Jane", "age": 25}}'
}

print("📊 Sample data created for tree visualization:")
for name, data in sample_data.items():
    print(f"   {name}: {len(data)} characters")
    print(f"   Preview: {data[:50]}...")
    print()

## 2. Generate Symbolic Trees

Now let's process each sample through FRACKTAL to create symbolic trees.

In [ ]:
# Initialize FRACKTAL
fracktal = RecursiveFRSOE(
    hash_depth=3,
    symbol_range=5000,
    min_pattern_length=3,
    min_occurrences=2
)

print("🚀 FRACKTAL initialized for tree visualization")
print(f"   - Hash depth: {fracktal.hash_depth}")
print(f"   - Symbol range: {fracktal.symbol_range}")
print(f"   - Min pattern length: {fracktal.recursive_compressor.min_pattern_length}")
print(f"   - Min occurrences: {fracktal.recursive_compressor.min_occurrences}")

In [ ]:
# Process each sample and extract symbolic trees
trees = {}
codexes = {}

for name, data in sample_data.items():
    print(f"\n🌳 Processing {name}...")
    
    # Compress the data
    compressed_result = fracktal.compress(data)
    
    # Extract the symbolic tree
    symbolic_tree = compressed_result['codex_map']['symbolic_tree']
    codex_map = compressed_result['codex_map']
    
    trees[name] = symbolic_tree
    codexes[name] = codex_map
    
    print(f"   📏 Original: {len(data)} characters")
    print(f"   🌿 Tree nodes: {symbolic_tree.node_count}")
    print(f"   📊 Max depth: {symbolic_tree.max_depth}")
    print(f"   🎯 Unique symbols: {codex_map.unique_symbols}")
    print(f"   🔗 Root nodes: {len(symbolic_tree.root_nodes)}")

print("\n✅ All symbolic trees generated successfully!")

## 3. Tree Structure Analysis

Let's analyze the structure of each symbolic tree.

In [ ]:
# Analyze tree structures
def analyze_tree_structure(tree):
    """Analyze the structure of a symbolic tree"""
    
    # Get depth distribution
    depth_counts = {}
    for node_data in tree.nodes.values():
        depth = node_data.get('depth', 0)
        depth_counts[depth] = depth_counts.get(depth, 0) + 1
    
    # Get link distribution
    link_counts = {}
    for node_data in tree.nodes.values():
        link = node_data.get('link', 'None')
        link_counts[link] = link_counts.get(link, 0) + 1
    
    return {
        'depth_distribution': depth_counts,
        'link_distribution': link_counts,
        'avg_depth': np.mean(list(depth_counts.keys())) if depth_counts else 0,
        'max_depth': max(depth_counts.keys()) if depth_counts else 0,
        'total_nodes': len(tree.nodes),
        'root_nodes': len(tree.root_nodes)
    }

# Analyze all trees
tree_analyses = {}
for name, tree in trees.items():
    print(f"\n📊 Analyzing {name} tree structure:")
    analysis = analyze_tree_structure(tree)
    tree_analyses[name] = analysis
    
    print(f"   🌿 Total nodes: {analysis['total_nodes']}")
    print(f"   📊 Max depth: {analysis['max_depth']}")
    print(f"   📈 Average depth: {analysis['avg_depth']:.2f}")
    print(f"   🌱 Root nodes: {analysis['root_nodes']}")
    print(f"   📋 Depth distribution: {dict(sorted(analysis['depth_distribution'].items()))}")

## 4. Interactive Tree Visualization

Now let's create interactive visualizations of the symbolic trees.

In [ ]:
def create_interactive_tree(tree, title="Symbolic Tree"):
    """Create an interactive visualization of a symbolic tree"""
    
    # Create NetworkX graph
    G = nx.DiGraph()
    
    # Add nodes and edges
    for symbol, data in tree.nodes.items():
        G.add_node(symbol, **data)
        if data.get('link'):
            G.add_edge(data['link'], symbol)
    
    # Calculate layout
    pos = nx.spring_layout(G, k=2, iterations=50)
    
    # Create edge traces
    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
    
    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=1, color='#888'),
        hoverinfo='none',
        mode='lines'
    )
    
    # Create node traces
    node_x = []
    node_y = []
    node_text = []
    node_colors = []
    node_sizes = []
    
    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)
        
        # Create hover text
        node_data = G.nodes[node]
        hover_text = f"Symbol: {node}<br>"
        hover_text += f"Depth: {node_data.get('depth', 0)}<br>"
        hover_text += f"Link: {node_data.get('link', 'None')}<br>"
        hover_text += f"Chunk: {node_data.get('chunk', 'N/A')}"
        node_text.append(hover_text)
        
        # Color by depth
        depth = node_data.get('depth', 0)
        node_colors.append(depth)
        
        # Size by connectivity
        size = 15 + len(list(G.predecessors(node))) * 5
        node_sizes.append(size)
    
    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers+text',
        hoverinfo='text',
        text=node_text,
        textposition="middle center",
        marker=dict(
            showscale=True,
            colorscale='Viridis',
            size=node_sizes,
            color=node_colors,
            colorbar=dict(
                thickness=15,
                title='Tree Depth',
                xanchor="left",
                len=0.5
            ),
            line_width=2
        )
    )
    
    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title=title,
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=20,l=5,r=5,t=40),
                        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
                    ))
    
    return fig

print("🎨 Creating interactive tree visualizations...")

# Create visualizations for each tree
for name, tree in trees.items():
    print(f"\n🌳 Creating visualization for {name}...")
    fig = create_interactive_tree(tree, f"{name.replace('_', ' ').title()} - Symbolic Tree")
    fig.show()
    
    # Also show tree statistics
    analysis = tree_analyses[name]
    print(f"   📊 Tree Statistics:")
    print(f"      Nodes: {analysis['total_nodes']}")
    print(f"      Max Depth: {analysis['max_depth']}")
    print(f"      Root Nodes: {analysis['root_nodes']}")

## 5. Tree Comparison Analysis

Let's compare the different tree structures to understand how different data types create different symbolic patterns.

In [ ]:
# Create comparison visualizations
print("📊 Creating tree comparison analysis...")

# Compare tree depths
fig_depth = go.Figure()
for name, analysis in tree_analyses.items():
    fig_depth.add_trace(go.Bar(
        name=name.replace('_', ' ').title(),
        x=['Max Depth', 'Avg Depth'],
        y=[analysis['max_depth'], analysis['avg_depth']],
        text=[f"{analysis['max_depth']}", f"{analysis['avg_depth']:.1f}"],
        textposition='auto'
    ))

fig_depth.update_layout(
    title="Tree Depth Comparison",
    xaxis_title="Depth Metric",
    yaxis_title="Depth",
    barmode='group'
)
fig_depth.show()

# Compare node counts
fig_nodes = go.Figure()
names = list(tree_analyses.keys())
node_counts = [tree_analyses[name]['total_nodes'] for name in names]
root_counts = [tree_analyses[name]['root_nodes'] for name in names]

fig_nodes.add_trace(go.Bar(
    name='Total Nodes',
    x=[name.replace('_', ' ').title() for name in names],
    y=node_counts,
    text=node_counts,
    textposition='auto'
))

fig_nodes.add_trace(go.Bar(
    name='Root Nodes',
    x=[name.replace('_', ' ').title() for name in names],
    y=root_counts,
    text=root_counts,
    textposition='auto'
))

fig_nodes.update_layout(
    title="Tree Node Comparison",
    xaxis_title="Data Type",
    yaxis_title="Node Count",
    barmode='group'
)
fig_nodes.show()

## 6. Depth Distribution Analysis

Let's examine how nodes are distributed across different depths in each tree.

In [ ]:
# Create depth distribution plots
fig_depth_dist = make_subplots(
    rows=2, cols=2,
    subplot_titles=[name.replace('_', ' ').title() for name in tree_analyses.keys()],
    specs=[[{"type": "bar"}, {"type": "bar"}],
           [{"type": "bar"}, {"type": "bar"}]]
)

colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']

for i, (name, analysis) in enumerate(tree_analyses.items()):
    row = (i // 2) + 1
    col = (i % 2) + 1
    
    depths = list(analysis['depth_distribution'].keys())
    counts = list(analysis['depth_distribution'].values())
    
    fig_depth_dist.add_trace(
        go.Bar(
            x=depths,
            y=counts,
            name=name.replace('_', ' ').title(),
            marker_color=colors[i],
            showlegend=False
        ),
        row=row, col=col
    )

fig_depth_dist.update_layout(
    title="Depth Distribution Across Tree Types",
    height=800
)

fig_depth_dist.update_xaxes(title_text="Depth", row=2)
fig_depth_dist.update_yaxes(title_text="Node Count", col=1)

fig_depth_dist.show()

print("📊 Depth distribution analysis completed!")

## 7. Key Insights from Tree Visualization

### What We've Learned:

🌳 **Tree Structure Patterns**:
- **Repetitive data** creates shallow, wide trees with many root nodes
- **Structured data** creates deeper trees with clear hierarchical patterns
- **Code-like data** creates complex trees with multiple branches
- **Nested data** creates deep, narrow trees with long chains

🔍 **Symbolic Intelligence**:
- FRACKTAL automatically identifies meaningful patterns
- Tree depth reflects data complexity
- Root nodes represent unique semantic units
- Link patterns show relationships between symbols

🎯 **Compression Insights**:
- Shallow trees = high compression potential
- Deep trees = complex semantic structure
- Root node count = semantic diversity
- Depth distribution = pattern complexity

### Applications:

• **Data Analysis**: Understand data structure through tree visualization
• **Pattern Recognition**: Identify repeating patterns in data
• **Compression Optimization**: Tune parameters based on tree structure
• **Semantic Analysis**: Explore meaning relationships in data

---

**FRACKTAL**: Where data becomes visual intelligence! 🌳✨